In [343]:
import requests
from requests.auth import HTTPBasicAuth
import os
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import pandas as pd
import re
pd.set_option('display.max_rows', 900)

bets = BETS()
criteria = ['U19 Gold','U19 National Championships','U17 National Championships','Senior Gold']
roundsWanted = ['SF','SF','Final','SF']
yongestTournament = 'U16'
matchData = get_tournament_results(bets,criteria,yongestTournament,roundsWanted,'16/09/2024')
# R128 ... QF SF Final
matchData


### H2H From all tournaments - no county - In Progress
#   Display tournament results in date order.
#   Extra tab for scores
#   Think of ways to tell the story. 
#   Flag unfinished matches
## Make into a Shiny App

u16
u15
u14
u13
u12
u11


,Tournament,Code,Winner,ScoreStatus,RoundName,EventName,Team1_MemberID,Team1_Firstname,Team1_Lastname,Team1_GenderID,Team2_MemberID,Team2_Firstname,Team2_Lastname,Team2_GenderID,Set_Scores,Min_Round,Prio
0,Kent U19 Gold 2024,198,1,0,R128,OS U19,[1350290],[Nived],[Neelakandan],[1],[1383941],[Deegallegedara Ravindu],[Sandaruwan],[1],"[{'Set_Team1': '21', 'Set_Team2': '8'}, {'Set_...",SF,primary
1,Kent U19 Gold 2024,164,1,0,R64,OS U19,[1300753],[Trevon],[Dias],[1],[1297827],[Krish],[Bodalia],[1],"[{'Set_Team1': '21', 'Set_Team2': '14'}, {'Set...",SF,primary
2,Kent U19 Gold 2024,247,2,0,R128,OS U19,[1323028],[Nikhil],[Pulikkottil],[1],[1288322],[Alex],[Lamb],[1],"[{'Set_Team1': '9', 'Set_Team2': '21'}, {'Set_...",SF,primary
3,Kent U19 Gold 2024,239,2,0,R128,OS U19,[1310400],[Hugo],[Dulson],[1],[1335295],[Omar],[Tariq],[1],"[{'Set_Team1': '14', 'Set_Team2': '21'}, {'Set...",SF,primary
4,Kent U19 Gold 2024,222,1,0,R128,OS U19,[1298434],[Jayaditya],[Kumar],[1],[1342747],[Charlie],[Qi],[1],"[{'Set_Team1': '21', 'Set_Team2': '11'}, {'Set...",SF,primary
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20447,Lancashire Junior 2023,162,1,0,R3,XD U15,[],"[Sanil, Ananya]","[Anirudh, Sanil]","[1, 2]",[],"[Felizian, Kathy]","[Merk, Su]","[1, 2]","[{'Set_Team1': '15', 'Set_Team2': '13'}, {'Set...",,secondary
20448,Lancashire Junior 2023,345,1,0,R3,XD U19,[],"[Matthew, Amalia]","[Jacks, Parker]","[1, 2]",[],"[Krish, Rebecca]","[Milan, Evans]","[1, 2]","[{'Set_Team1': '15', 'Set_Team2': '10'}, {'Set...",,secondary
20449,Lancashire Junior 2023,166,1,0,R3,XD U15,[],"[Dylan, Dina]","[Huynh, Yuetong]","[1, 2]",[],"[Ethan, Stella]","[Shou, Ren]","[1, 2]","[{'Set_Team1': '12', 'Set_Team2': '15'}, {'Set...",,secondary
20450,Lancashire Junior 2023,329,1,0,R3,XD U19,[],"[Harley, Sarah]","[Edge, Fisher]","[1, 2]",[],"[Bharath, Freya]","[Cheralathan, Davey]","[1, 2]","[{'Set_Team1': '15', 'Set_Team2': '3'}, {'Set_...",,secondary


In [344]:
class BETS():
    def __init__(self):
        load_dotenv()
        self.clientId = os.getenv('CLIENT_ID')
        self.clientPw = os.getenv('CLIENT_PW')
        self.baseUrl = "https://api.tournamentsoftware.com/1.0"
    
    def make_call(self,url):
        response = requests.get(url, auth=HTTPBasicAuth(self.clientId, self.clientPw))
        return response

    #   USE SEARCH QUERY '?q=U11 Gold'
    #   Filter by dates
    #   Create lists of cool tourney codes.
    def search_tournaments(self,query):
        outputQuery = query.replace(' ','+')
        if query:
            url = self.baseUrl+f"/Tournament?q={outputQuery}"
        else:
            yearAgo = datetime.strftime(datetime.today() - timedelta(days=365),'%Y-%m-%d')
            url = self.baseUrl+f"/Tournament?list=1&refdate={yearAgo}&pagesize=1000" 
        return self.make_call(url)
    
    def get_tournament_details(self,tCode):
        url = self.baseUrl+f"/Tournament/{tCode}"
        return self.make_call(url)

    def get_matches(self,tCode,date):
        dateQuery = date.strftime("%Y%m%d")
        url = self.baseUrl+f"/Tournament/{tCode}/Match/{dateQuery}"
        return self.make_call(url)
    
    def wanted_rounds(self,roundCaredAbout):
        allRounds = ['R128', 'R64', 'R32', 'R16', 'QF', 'SF', 'Final']
        if roundCaredAbout in allRounds:
            index = allRounds.index(roundCaredAbout)
            return allRounds[index:]
        else:
            return ['','','','','','','']


In [345]:
def processXMLList(list):
    out = []
    for xml in list:
        out.append(xml.text)
    return out

def exclude_list(inputs,youngestTournament):
    output = ['invite','(1000ie)','masters','senior county','para','yonex','league','regional','schools','graded','rising']
    startingAge = int(youngestTournament[1:])
    if startingAge > 13:
        output.append('futures')        
    for i in range(startingAge,10,-1):
        print(f'u{i}')
        output.append(f'u{i}')
        
    return output
   
#Both inputs as strings, searchStart = d/m/y, searchText = 'U11 Gold'
def get_tournament_results(bets,searchText,youngestTournament,roundsWanted,searchStart):
    tabuList = exclude_list(searchText,youngestTournament)
    searchStart = datetime.strptime(searchStart,"%d/%m/%Y")
    tourneysDf = pd.DataFrame(columns=['Name','Code','StartDate','EndDate'])
    tourneysToAdd = []
    searchText.append('')
    roundsWanted.append('')
    for run,search in enumerate(searchText):

        if not search: #tournaments are for h2h only, not for selection
            tournamentPrio = 'secondary'
        else:
            tournamentPrio = 'primary'

        out = bets.search_tournaments(search).content
        soup = BeautifulSoup(out,'xml')
        tournaments = soup.find_all('Tournament')

        for i in range(0,len(tournaments)):  
            name = tournaments[i].find('Name').text
            code = tournaments[i].find('Code').text
            startDate = datetime.fromisoformat(tournaments[i].find('StartDate').text)
            endDate = datetime.fromisoformat(tournaments[i].find('EndDate').text)

            tType = tournaments[i].find('TypeID').text
            
            goodTourney = True

            if not tType == '0':
                goodTourney = False
            
            for t in tabuList:
                if t in name.lower():
                    goodTourney = False

            if not any(existing_tourney['Name'] == name for existing_tourney in tourneysToAdd) and goodTourney:
                tourney = {
                    "Name": name,
                    "Code": code,
                    "StartDate": startDate,
                    "EndDate": endDate,
                    "Min_Round": roundsWanted[run],
                    "Prio": tournamentPrio
                }
    
                tourneysToAdd.append(tourney)

    tourneysDf = pd.concat([tourneysDf, pd.DataFrame(tourneysToAdd)], ignore_index=True)
    if searchStart < datetime.today():
        searchDate = searchStart
    else:
        searchDate = datetime.today() - timedelta(days = 2)

    # Define searchDate and dtYearBefore
    dtYearBefore = searchDate - timedelta(days=365)
    
    # Apply condition and filter rows
    tourneysInYearDf = tourneysDf[(tourneysDf['EndDate'] >= dtYearBefore) & (tourneysDf['StartDate'] <= searchDate) & (tourneysDf['EndDate'] < searchDate)].copy()

    #Itterate through Tournaments
    match_data = []
    for index, row in tourneysInYearDf.iterrows():
        roundWanted = row['Min_Round']
        dtCurrentDay = row['StartDate']
        dtEndDate = row['EndDate']
        code = row['Code']
        tName = row['Name']
        tPrio = row['Prio']
        
        #Itterate through Days With Matches.
        allMatchesXML = [] 
        while dtCurrentDay <= dtEndDate:
            tDetails = bets.get_matches(row['Code'],dtCurrentDay)
            matchesXML = tDetails.content
            allMatchesXML.append(matchesXML)
            dtCurrentDay = dtCurrentDay + timedelta(days=1)

        # Iterate over each Match element
        for matchesXML in allMatchesXML:
            soup = BeautifulSoup(matchesXML,'xml')
            for match in soup.find_all('Match'):
                #check if match went ahead:
                if not match.find('ScoreStatus').text == '0':
                    pass
                else:
                    match_info = {
                        'Tournament': tName,
                        'Code': match.find('Code').text,
                        'Winner': match.find('Winner').text,
                        'ScoreStatus': match.find('ScoreStatus').text,
                        'RoundName': match.find('RoundName').text,
                        'EventName': match.find('EventName').text,
                    }
                    # Extract team and player details
                    teams = []
                    for team in ['Team1', 'Team2']:
                        teamXML = match.find(team)
                        team_info = {}
                        #find player id's and if singles
                        team_info[f'{team}_MemberID'] = processXMLList(teamXML.find_all('MemberID'))
                        team_info[f'{team}_Firstname'] = processXMLList(teamXML.find_all('Firstname'))
                        team_info[f'{team}_Lastname'] = processXMLList(teamXML.find_all('Lastname'))
                        team_info[f'{team}_GenderID'] = processXMLList(teamXML.find_all('GenderID'))

                        teams.append(team_info)

                    match_info.update(teams[0])
                    match_info.update(teams[1])

                    # Extract sets scores
                    sets = match.find('Sets')
                    set_scores = []
                    if sets:
                        for set_ in sets.find_all('Set'):
                            set_scores.append({
                                'Set_Team1': set_['Team1'],
                                'Set_Team2': set_['Team2']
                            })
    
                        match_info['Set_Scores'] = set_scores
                    
                    match_info['Min_Round'] = roundWanted

                    match_info['Prio'] = tPrio

                    # Append the structured match information to the list
                    match_data.append(match_info)

            # Convert the list to a DataFrame
    df = pd.DataFrame(match_data)
    return df

def next_func():
    pass

In [346]:
class Player():
    def __init__(self,name,id,event):
        self.name = name
        self.id = id
        self.event = event
        self.playerNamesBeat = []
        self.playerIDsBeat = []
        self.h2h = []
    
    def display(self):
        print(f'{self.name} ({self.id}) : {self.playersBeat}')
    
def clean_event(event):
    if 'open' in event.lower():
        if 'd' in event.lower():
            event = 'md'
        else:
            event = 'ms'

    elif 'ladi' in event.lower():
        if 'd' in event.split(' ')[1].lower():
            event = 'wd'
        else:
            event = 'ws'

    elif 'men' in event.lower():
        if 'd' in event.lower():
            event = 'md'
        else:
            event = 'ms'
    elif 'mixe' in event.lower():
        event = 'xd'
    
    elif 'boy' in event.lower():
        if 'd' in event.lower():
            event = 'md'
        else:
            event = 'ms'
    
    elif 'girl' in event.lower():
        if 'd' in event.lower():
            event = 'wd'
        else:
            event = 'ws'

    elif 'mx' in event.lower():
        event = 'xd'

    else:
        event = event.lower().replace('o','m').replace('g','w').replace('b','m').replace('*','').split(' ')[0].split('u')[0]
    
    return event

def check_for_ids(event,t1ID,t2ID):
    if 'd' in event:
        if (not len(t1ID) > 1) or (not len(t2ID) > 1): 
            return False
        for id_ in t1ID:
            if not id_:
                return False
        for id_ in t2ID:
            if not id_:
                return False
        return True
    else:
        if t1ID and t2ID:
            return True
    return False
        
def add_match(tournament,event,winningTeam,t1name,t1ID,t2name,t2id,scores,code):
    pass

likelyPlayersDf = pd.read_csv('likely_players.csv')
likelyPlayersDf['Beat'] = [[] for _ in range(len(likelyPlayersDf))]
likelyPlayersDf['Temp'] = ''
primaryTournaments = matchData[matchData['Prio'] == 'primary']
uniqueTourneys = primaryTournaments['Tournament'].unique()
for tName in uniqueTourneys:
    for index, row in likelyPlayersDf.iterrows():
        if 's' in likelyPlayersDf.at[index, 'Event']:
            likelyPlayersDf[tName] = ''
        else:
            likelyPlayersDf[tName] = ' - '

msDf = likelyPlayersDf[likelyPlayersDf['Event'] == 'ms']  # Men's Singles
wsDf = likelyPlayersDf[likelyPlayersDf['Event'] == 'ws']  # Women's Singles
mdDf = likelyPlayersDf[likelyPlayersDf['Event'] == 'md']  # Men's Doubles
wdDf = likelyPlayersDf[likelyPlayersDf['Event'] == 'wd']  # Women's Doubles

xdDf = likelyPlayersDf[likelyPlayersDf['Event'] == 'xd']  # Mixed Doubles
mxdDf = xdDf.iloc[:len(xdDf)//2]
wxdDf = xdDf = xdDf.iloc[len(xdDf)//2:]

eventDfList = [msDf,wsDf,mdDf,wdDf,mxdDf,wxdDf]

matchesDf = pd.DataFrame()

def get_event_df(event,eventDfList,gender = 0):
    if event == 'ms':
        return(eventDfList[0])
    if event == 'ws':
        return(eventDfList[1])
    if event == 'md':
        return(eventDfList[2])
    if event == 'wd':
        return(eventDfList[3])
    if event == 'mxd':
        return(eventDfList[4])
    if event == 'wxd':
        return(eventDfList[5])


bets = BETS()
for index, row in matchData.iterrows():
    round = row['RoundName']
    roundsWanted = bets.wanted_rounds(row['Min_Round'])
    tournament = row['Tournament']
    event = clean_event(row['EventName'])
    prio = row['Prio']        
    winningTeam = row['Winner']
    t1ID = row['Team1_MemberID']
    t2ID = row['Team2_MemberID']
    idsExist = check_for_ids(event,t1ID,t2ID)
    t1Gender = row['Team1_GenderID']
    t2Gender = row['Team2_GenderID']
    code = row['Code']

    if 's' in event:
        t1Name = (str(row['Team1_Firstname']) + ' ' + str(row['Team1_Lastname'])).replace("'",'').replace('[','').replace(']','')
        t2Name = (str(row['Team2_Firstname']) + ' ' + str(row['Team2_Lastname'])).replace("'",'').replace('[','').replace(']','')
    else:
        teamNames = []
        for team in ['Team1','Team2']:
            names = []
            for i in range(0,2):
                name = str((row[f'{team}_Firstname'][i]) + ' ' + str(row[f'{team}_Lastname'][i])).replace("'",'').replace('[','').replace(']','')
                names.append(name)
            teamNames.append(names)
        t1Name = teamNames[0]
        t2Name = teamNames[1]
    scores = row['Set_Scores']

    eventDf = get_event_df(event,eventDfList)

    if idsExist:
        if 's' in event: # Singles
            if str(winningTeam) == '1':
                winnerID = str(t1ID)[2:-2]
                winnerName = t1Name
                looserID = str(t2ID)[2:-2]
                looserName = t2Name
            else:
                winnerID = str(t2ID)[2:-2]
                winnerName = t2Name
                looserID = str(t1ID)[2:-2]  
                looserName = t1Name    

            ids = [winnerID,looserID]
            
            for i,id in enumerate(ids):
                matching_row = eventDf[(eventDf['ID'] == int(id)) & (eventDf['Event'] == event)] #Find Winner
                if not matching_row.empty:
                    # Get the index of the matching row
                    idx = matching_row.index[0]
                    # Append the t2ID to the 'Beat' list in the matching row
                    if i == 0 and not (eventDf[eventDf['ID'] == int(ids[1])].empty):
                        eventDf.at[idx, 'Beat'].append(ids[1])
                    #Find best round
                    if (row['RoundName'] in str(roundsWanted)) and prio == 'primary':
                        currentBest = eventDf.at[idx,tournament]
                        if len(bets.wanted_rounds(currentBest)) >= len(bets.wanted_rounds(round)):

                            if i == 0:
                                eventDf.at[idx, 'Temp'] = 'win'
                            else:
                                eventDf.at[idx, 'Temp'] = 'loose'   

                            if currentBest == 'Winner':
                                pass
                            else:
                                if round == 'Final' and i == 0:
                                    eventDf.at[idx,tournament] = 'Winner'
                                else:
                                    eventDf.at[idx,tournament] = round
        elif event == 'xd':   #Mixed Dubs
            runs = ['mxd','wxd']
            for run in range(0,2):
                eventDf = get_event_df(runs[run],eventDfList)
                if str(winningTeam) == '1':
                    winnerID = str(t1ID[run])
                    winnerName = t1Name[run]
                    looserID = str(t2ID[run])
                    looserName = t2Name[run]
                else:
                    winnerID = str(t2ID[run])
                    winnerName = t2Name[run]
                    looserID = str(t1ID[run])
                    looserName = t1Name[run]      

                ids = [winnerID,looserID]
                for i,id in enumerate(ids):
                    matching_row = eventDf[(eventDf['ID'] == int(id)) & (eventDf['Event'] == event)] #Find Winner
                    if not matching_row.empty:
                        # Get the index of the matching row
                        idx = matching_row.index[0]
                        # Append the t2ID to the 'Beat' list in the matching row
                        if i == 0 and not (eventDf[eventDf['ID'] == int(ids[1])].empty):
                            eventDf.at[idx, 'Beat'].append(ids[1])
                        #Find best round
                        if (row['RoundName'] in str(roundsWanted)) and prio == 'primary':
                            currentBest = eventDf.at[idx,tournament].split(' - ')[0]
                            if len(bets.wanted_rounds(currentBest)) >= len(bets.wanted_rounds(round)):      
                                if run == 0 and winningTeam == '1':
                                    partner = f'{t1Name[1]} ({t1ID[1]})'
                                elif run == 0 and winningTeam == '2':
                                    partner = f'{t2Name[1]} ({t2ID[1]})'
                                elif run == 1 and winningTeam == '1':
                                    partner = f'{t1Name[0]} ({t1ID[0]})'
                                elif run == 1 and winningTeam == '2':
                                    partner = f'{t2Name[0]} ({t2ID[0]})'

                                if i == 0:
                                    eventDf.at[idx, 'Temp'] = 'win'
                                else:
                                    eventDf.at[idx, 'Temp'] = 'loose'   

                                if currentBest == 'Winner':
                                    pass
                                else:
                                    if round == 'Final' and i == 0:
                                        eventDf.at[idx,tournament] = 'Winner - ' + partner
                                    else:
                                        eventDf.at[idx,tournament] = round + ' - ' + partner

        else:
            if str(winningTeam) == '1':
                winnerIDs = t1ID
                winnerNames = t1Name
                looserIDs = t2ID
                looserNames = t2Name
            else:
                winnerIDs = t2ID
                winnerNames = t2Name
                looserIDs = t1ID
                looserNames = t1Name

            ids = [winnerIDs,looserIDs]
            names = [winnerNames,looserNames]
            for i,id in enumerate(ids):
                for memberNum,memberID in enumerate(id):
                    matching_row = eventDf[(eventDf['ID'] == int(memberID)) & (eventDf['Event'] == event)] #Find Winner
                    if not matching_row.empty:
                        # Get the index of the matching row
                        idx = matching_row.index[0]
                        # Append the t2ID to the 'Beat' list in the matching row
                        if i == 0 and not (eventDf[eventDf['ID'] == int(ids[1][0])].empty):
                            eventDf.at[idx, 'Beat'].append(ids[1][0])
                        if i == 0 and not (eventDf[eventDf['ID'] == int(ids[1][1])].empty):
                            eventDf.at[idx, 'Beat'].append(ids[1][1])

                        #Find best round
                        if (row['RoundName'] in str(roundsWanted)) and prio == 'primary':
                            currentBest = eventDf.at[idx,tournament].split(' - ')[0]
                            if len(bets.wanted_rounds(currentBest)) >= len(bets.wanted_rounds(round)):
                                if memberNum == 1:
                                    partner = f'{names[i][0]} ({id[0]})'
                                else:
                                    partner = f'{names[i][1]} ({id[1]})'

                                if i == 0:
                                    eventDf.at[idx, 'Temp'] = 'win'
                                else:
                                    eventDf.at[idx, 'Temp'] = 'loose' 

                                if currentBest.split(' - ') == 'Winner':
                                    pass
                                else:
                                    if round == 'Final' and i == 0:
                                        eventDf.at[idx,tournament] = 'Winner - ' + partner
                                    else:
                                        eventDf.at[idx,tournament] = round + ' - ' + partner

        for index, row in eventDf.iterrows():
            if prio == 'primary':
                try:
                    places = row[tournament].split(' - ')[0]
                    partner = row[tournament].split(' - ')[1]
                except:
                    places = row[tournament]
                if not (places == 'Final' or places == 'Winner') and places:
                    if row['Temp'] == 'win':
                        if partner:
                            eventDf.at[index, tournament] = str(bets.wanted_rounds(places)[1]) + ' - ' + partner
                        else:
                            eventDf.at[index, tournament] = bets.wanted_rounds(places)[1]

    #if (('Benjamin' in row['Name']) and event == 'xd'):
    #    print(tournament)
    #    print(t1Name)
    #    print(t2Name)
    #    print(t1ID)
    #    print(event)
    #    print(round)
    #    print('')

        

Jason beat Oli
Derby University U19 Gold SF ms
Jason beat Oli
U19 English National Championships 2023 R16 ms


In [347]:
pd.set_option('display.max_rows', 5)
eventDfList[3]

,Name,Event,ID,Beat,Temp,Kent U19 Gold 2024,Derby University U19 Gold,U19 English National Championships 2023,U17 English National Championships 2023,Dorset Senior Gold 2024,Nottinghamshire Senior Gold 2024,The Middlesex Senior Gold Tournament 2023
63,Lucy Brierley,wd,1286589,"[1321479, 1293518, 1323320, 1254091, 1321479, ...",win,Final - Matilda Franklin (1270523),Final - Matilda Franklin (1270523),Final - Sofie Chong (1293518),Winner - Sofie Chong (1293518),-,-,-
64,Carly Stevenson,wd,1254091,"[1323877, 1336134, 1325825, 1274309, 1317894, ...",loose,SF - Saffron Morris (1291891),-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...
82,Anoushka Shetty,wd,1323877,"[1322371, 1297947]",,-,-,-,-,-,-,-
83,Rhea Ramakrishnan,wd,1267758,[1321985],,-,-,-,-,-,-,-


In [348]:
targetPlayersDfs = []
for df in eventDfList:
    cols_after_5 = df.iloc[:, 5:]
    # Create a mask to check if any value in the row is not equal to "-"
    mask = (cols_after_5 != " - ").any(axis=1)
    # Filter the original DataFrame based on the mask
    PlayersToTrack = df[mask]
    targetPlayersDfs.append(PlayersToTrack)

targetPlayersDfs[3]

,Name,Event,ID,Beat,Temp,Kent U19 Gold 2024,Derby University U19 Gold,U19 English National Championships 2023,U17 English National Championships 2023,Dorset Senior Gold 2024,Nottinghamshire Senior Gold 2024,The Middlesex Senior Gold Tournament 2023
63,Lucy Brierley,wd,1286589,"[1321479, 1293518, 1323320, 1254091, 1321479, ...",win,Final - Matilda Franklin (1270523),Final - Matilda Franklin (1270523),Final - Sofie Chong (1293518),Winner - Sofie Chong (1293518),-,-,-
64,Carly Stevenson,wd,1254091,"[1323877, 1336134, 1325825, 1274309, 1317894, ...",loose,SF - Saffron Morris (1291891),-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...
75,Mia Fox,wd,1325825,"[1317894, 1312773, 1312773, 1321479, 1308656, ...",loose,-,SF - Yashica Racharla (1308656),-,-,-,-,-
78,Anna Stewart,wd,1322371,[1297947],loose,SF - Mili Patel (1312773),-,-,-,-,-,-


In [349]:
h2hMatrix = []
for df in targetPlayersDfs:
    unique_ids = df['ID'].unique()  # Get all unique player IDs
    head_to_head_matrix = pd.DataFrame('-', index=unique_ids, columns=unique_ids)

    # Create a dictionary to store the counts for both wins and losses
    head_to_head_counts = {id_: {id2: [0, 0] for id2 in unique_ids} for id_ in unique_ids}

    playerIds = df['ID'].tolist()

    # Fill in the counts based on the Beat column
    for idx, row in df.iterrows():
        winner_id = row['ID']  # The player who won
        beaten_ids = row['Beat']  # The players they beat
        for beaten_id in beaten_ids:
            if beaten_id in str(playerIds):
                head_to_head_counts[int(winner_id)][int(beaten_id)][0] += 1  # Increment winner's wins
                head_to_head_counts[int(beaten_id)][int(winner_id)][1] += 1  # Increment beaten player's losses

    # Convert the counts into the "Wins-Losses" format
    for winner_id in unique_ids:
        for opponent_id in unique_ids:
            wins, losses = head_to_head_counts[winner_id][opponent_id]
            if wins > 0 or losses > 0:
                head_to_head_matrix.loc[winner_id, opponent_id] = f"{wins}-{losses}"

    # Create a mapping from IDs to names
    id_to_name = dict(zip(df['ID'], df['Name']))

    # Replace the index and columns of the head-to-head matrix with player names
    head_to_head_matrix = head_to_head_matrix.rename(index=id_to_name, columns=id_to_name)

    # Display the head-to-head matrix with names instead of IDs
    head_to_head_matrix.columns.name = 'Beaten Player'
    head_to_head_matrix.index.name = 'Winning Player'
    h2hMatrix.append(head_to_head_matrix)


In [350]:
h2hMatrix[0]

Beaten Player,Jason Ou,Oliver Nicolson,Oliver Wu,James Song,Daniel Wykes,Sonish Hathi
Winning Player,,,,,,
Jason Ou,-,1-0,2-0,-,2-0,-
Oliver Nicolson,0-1,-,0-1,-,-,-
...,...,...,...,...,...,...
Daniel Wykes,0-2,-,-,1-0,-,-
Sonish Hathi,-,-,0-1,0-1,-,-


In [351]:
import pandas as pd
from openpyxl import load_workbook

outputDfs = []
for df in targetPlayersDfs:
    df = df.drop(['Beat', 'Temp', 'Event'], axis=1)
    outputDfs.append(df)

dfHeaddigns = ['MS','WS','MD','WD','MXD','WXD']

with pd.ExcelWriter('script-results.xlsx', engine='xlsxwriter') as writer:

    outputDfs[0].to_excel(writer, sheet_name='Tournaments', startrow=1, index=False)  
    # Access the worksheet object created by writer
    worksheet = writer.sheets['Tournaments']
    # Add a heading for the first DataFrame at the top
    worksheet.write(0, 0, 'MS')  # Write heading

    # Write first dataframe with heading
    workbook = writer.book
    worksheet = writer.sheets['Tournaments']
    row_position = len(outputDfs[0]) + 3
    for i in range(1,len(outputDfs)):
        worksheet.write(row_position, 0, dfHeaddigns[i])  # Add a heading for the first DataFrame
        outputDfs[i].to_excel(writer, sheet_name='Tournaments', startrow=row_position + 1, index=False)
        row_position += len(outputDfs[i]) + 3
    
    h2hMatrix[0].to_excel(writer, sheet_name='H2H', startrow=1)  
    # Access the worksheet object created by writer
    worksheet = writer.sheets['H2H']
    # Add a heading for the first DataFrame at the top
    worksheet.write(0, 0, 'MS')  # Write heading
    
    workbook = writer.book
    worksheet = writer.sheets['H2H']
    row_position = len(h2hMatrix[0]) + 3
    for i in range(1,len(h2hMatrix)):
        worksheet.write(row_position, 0, dfHeaddigns[i])  # Add a heading for the first DataFrame
        h2hMatrix[i].to_excel(writer, sheet_name='H2H', startrow=row_position + 1)
        row_position += len(h2hMatrix[i]) + 3